# PRE-PROCESSING:

In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [5]:
def importa_dati(path, sep=','):
    """Funzione per l'importazione di un dataset con un formato csv in un DataFrame di Pandas.
       - path: percorso in cui è salvato il file.csv (stringa);
       - sep: separatore utilizzato nel file (stringa)"""
    data = pd.read_csv(path, sep=sep)
    data.drop('Unnamed: 0', axis=1, inplace=True)
    return data

In [6]:
# importazione dataset multiclasse
#df = importa_dati('/Users/eliaceccolini/Documents/Uni/Tesi/Dataset_finale/Datasets/Dataset_analizzato_multiclasse.csv')
# importazione dataset binario
df = importa_dati('/Users/eliaceccolini/Documents/Uni/Tesi/Dataset_finale/Datasets/Dataset_analizzato_binario.csv')

***
### Sostituzione del peso con BMI:

In [7]:
def calcola_BMI(data, colonna_peso, colonna_alt, nome_colonna_bmi):
    """Funzione per il calcolo del BMI. Inoltre le colonne di peso e altezza verranno sostituite da questo valore.
       - data: dataset (DataFrame)
       - colonna_peso: nome colonna peso (string)
       - colonna_alt: nome colonna altezza (string)
       - nome_colonna_bmi: nome nuova colonna"""
    data[colonna_peso] = [float((p.replace(",", "."))) if not isinstance(p, float) else p for p in data[colonna_peso]]
    data[colonna_alt] = [float(p) if not isinstance(p, float) else p for p in data[colonna_alt]]
    data[nome_colonna_bmi] = round(data[colonna_peso] / ((data[colonna_alt] / 100) ** 2), 2)
    data.drop(colonna_peso, axis=1, inplace=True)
    data.drop(colonna_alt, axis=1, inplace=True)

In [8]:
cols_peso = ['somm1_peso', 'somm2_peso', 'somm3_peso', 'somm4_peso', 'somm5_peso', 'somm6_peso']
cols_altezza = ['somm1_altezza', 'somm2_altezza', 'somm3_altezza', 'somm4_altezza', 'somm5_altezza', 'somm6_altezza']
num_somm = ['1', '2', '3', '4', '5', '6']
for i in range(len(num_somm)):
    calcola_BMI(df, cols_peso[i], cols_altezza[i], 'somm'+num_somm[i]+'_BMI')

***
### Trasformazione dei dati categoriali utilizzando la tecnica del One Hot Encoding:

In [9]:
def trasforma_categoriali_OHE(data):
    """Funzione per trasformare le colonne in tipo categoriale codificandole tramite OHE.
       - data: dataset (DataFrame)"""
    values_names = ['creatinina', 'creatinina2', 'glucosio', 'emoglobina']
    sympt_names = ['fatigue', 'pain', 'nausea', 'diarrea']
    cols_values = []
    cols_sympt = []
    for name in values_names:
        cols_values = cols_values + ['somm'+str(i)+'_'+name for i in range(1, 7)]
    for name in sympt_names:
        cols_sympt = cols_sympt + ['somm'+str(i)+'_'+name for i in range(1, 7)]
    cols_cat = ['sesso', 'consumatore_alcool', 'fumatore', 'gruppo_patologia', 'Patologia', 'classe_eta', 'classi_principi_attivi', 't', 'n', 'm']
    cols_cat = cols_values + cols_cat
    cols_cat = cols_sympt + cols_cat
    cols_num = list(set(data.columns) - set(cols_cat))
    
    OHenc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    new_cat_data = pd.DataFrame(OHenc.fit_transform(data[cols_cat]), dtype=int)
    new_cat_data['index'] = new_cat_data.index
    data.drop(cols_cat, axis=1, inplace=True)
    data['index'] = data.index
    new_data = pd.merge(data, new_cat_data, on='index')
    new_data.drop('index', axis=1)
    return new_data

In [10]:
# utilizzando OHE
df = trasforma_categoriali_OHE(df)
#df.shape

***
### Esportazione dei dati:

In [11]:
def esporta_dati(data, path):
    """Funzione per il salvataggio del dataset in un file csv.
       - data: dataset (DataFrame)
       - path: percorso in cui salvare i dati""" 
    data.to_csv(path)

In [12]:
esporta_dati(df, '/Users/eliaceccolini/Documents/Uni/Tesi/Dataset_finale/Datasets/Dataset_preprocessato_OHE.csv')